In [1]:
import spacy
import scispacy
from scispacy.linking import EntityLinker

nlp = spacy.load("en_core_sci_sm")
nlp.add_pipe ("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 3})

c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfTransformer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\sklearn\base.py:338: UserWarning: Trying to unpickle estimator TfidfVectorizer from version 0.20.3 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,


In [2]:
import requests, re
from bs4 import BeautifulSoup

MATCH_ARTICLE_BODY = [re.compile('.*RichTextContainer.*')]

list_of_extracted_texts = []

SITEMAP = "https://www.bbc.co.uk/sitemaps/https-sitemap-uk-archive-42.xml"
req = requests.get(SITEMAP)

match = re.findall("<loc>(.*health.*)</loc>",req.text)

for url in match[:50]:
    req = requests.get(url)
    soup = BeautifulSoup(req.content, 'html.parser')

    text_as_list = [
        el.text for el
        in soup.find_all("div", MATCH_ARTICLE_BODY, recursive=True)
    ]

    text_as_string = "".join(text_as_list)
    list_of_extracted_texts.append(text_as_string)

In [3]:
for text in list_of_extracted_texts[:50]:
    doc = nlp(text)
    fmt_str = "{:20} | {:11} | {:6}"
    print(fmt_str.format("Entity", "Concept ID", "Score"))
    entity = doc.ents[2]
    for kb_entry in entity._.kb_ents:
        cui = kb_entry[0]
        match_score = kb_entry[1]
        print(fmt_str.format(entity.text, cui, match_score))

c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\scispacy\candidate_generation.py:284: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_neighbors[empty_vectors_boolean_flags] = numpy.array(neighbors)[:-1]
c:\Apps\Anaconda\envs\BBC Health data analysis\lib\site-packages\scispacy\candidate_generation.py:285: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  extended_distances[empty_vectors_boolean_flags] = numpy.array(distances)[:-1]


Entity               | Concept ID  | Score 
severe hepatitis     | C0019158    | 0.7381238341331482
severe hepatitis     | C0019159    | 0.7381238341331482
Entity               | Concept ID  | Score 
England              | C0014282    |    1.0
England              | C5205018    | 0.8346024751663208
England              | C0027965    | 0.7653471827507019
Entity               | Concept ID  | Score 
million              | C1881839    |    1.0
million              | C0439187    | 0.8909984827041626
million              | C1881845    | 0.7906677722930908
Entity               | Concept ID  | Score 
suicide prevention   | C0204732    |    1.0
suicide prevention   | C0038661    | 0.7436481714248657
Entity               | Concept ID  | Score 
choose supermarket value | C0557778    | 0.7087706923484802


In [6]:
for text in list_of_extracted_texts[:50]:
    doc = nlp(text)
    linker = nlp. get_pipe("scispacy_linker")
    fmt_str = "{:<20} | {:<32}"
    print(fmt_str.format("Entity","Canonical Name"))
    for entity in doc.ents:
        if entity._.kb_ents:
            kb_entry = linker.kb.cui_to_entity[entity._.kb_ents[0][0]]
            print(fmt_str.format(entity.text, kb_entry.canonical_name))


Entity               | Canonical Name                  
cases                | Case (situation)                
children             | Child                           
severe hepatitis     | Hepatitis                       
detected             | Detected (finding)              
countries            | Country                         
South East Asia      | Asia, Southeastern              
World Health Organization | World Health Organization       
WHO                  | World Health Organization       
world                | World                           
investigating        | Investigates                    
mysterious           | RNF213 gene                     
increase             | Increase                        
cases                | Case (situation)                
liver condition      | Disease                         
virus                | Virus                           
adenovirus           | Adenoviruses                    
pandemic             | Pandemics           